In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import gpflow
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
from shgp import SHGP
import plotly.io as pio
pio.renderers.default = 'jupyterlab'
import plotly
from tqdm import trange
from helper_funs import add_plot, plot_clim, plot_fig3, fit_fig3, plot_fig5, fit_fig5

# Hide warnings
import warnings
warnings.filterwarnings(action='ignore')

# Seeding for reproducibility
SEED = 30
rng = np.random.RandomState(SEED)
tfp_seed = tfp.random.sanitize_seed(SEED)

# Plotly specifics
import plotly.io as pio
import plotly.graph_objects as go
pio.templates["pres"] = go.layout.Template(
    layout=go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        colorway=px.colors.qualitative.Set2,
        legend=dict(itemsizing='trace', font_size=26),
        font_size=26,\
        
    )
)

config= dict(displayModeBar=False)

# Set plotly defaults
pio.templates.default = 'none+pres'
full_fig_width = 1000
full_fig_height = 600
half_fig_width = full_fig_width // 2
half_fig_height = full_fig_height // 2

# Estimating latent processes from sets of vectors with sparse hierarchical Gaussian processes

## Identifying the underlying climate signals from multiple climate models

Dry title but I wanted to highlight that this work was about both methodological advancements and an important application.

Work done alongside DSNE PhD Tom Pinder, which forms part of a larger effort to construct climate projections from multiple climate models using Bayesian methods.

Say we had output from 5 different climate models which estimate the global mean surface temperature from 1960-2014. 

We would expect that all the models have a similar latent functionality which contains to smooth increase in temperature as the impact of anthropogenic emissions kicks in, coupled with the annual oscillation we see across all parts of the earth. But on top of that, because we are modelling such a chaotic system, each climate model has its own variability caused by meteorology and other subdecadal effects which creates the difference at in models across time.

I'm going to show you how we can model data like this in order to extract the latent function whilst making robust uncertainty estimates. I'll talk through this climate example, but bear in mind that we could also apply this method to problems such as modelling the latent air quality of a city from numerous sensors, or any time we want to model a hierarchical system.

In [ ]:
da = xr.open_dataarray('./../data/climate_model_data_1D_GMST.nc')
plot_clim(da)

### Hierarchical model
$$\mathbf{Y} = \{\mathbf{y}_1, \mathbf{y}_2, \mathbf{y}_3\}$$

$$\mathbf{y}_i = f_i(\mathbf{x}) + g(\mathbf{x})$$

In [ ]:
# Hide, Run initially
fig2 = go.FigureWidget()
fig2.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig2.update_layout(xaxis={'visible': False, 'showticklabels': False})

fig2

The way we want to model this data is hierarchically. Say for example we suppose that some times series y all share some latent functionality g, in addition to their individual functionality f_i.

You can see that here. We've defined the shared funcionality g and modulated it to create three different time series.

In [ ]:
x = np.linspace(-5, 5, 100)
g = np.sin(2 * x) + np.cos(1 * x)
add_plot(x, g, 'g(x)', fig2);

In [ ]:
y1 = g + np.sin(3 * x)
add_plot(x, y1, 'y1', fig2)
y2 = g + x * 0.2
add_plot(x, y2, 'y2', fig2)
y3 = g + 1
add_plot(x, y3, 'y3', fig2)


\begin{align}
g(\mathbf{x}) &\sim \mathcal{GP}(\mathbf{0}, k_g(\mathbf{x}, \mathbf{x}')) \\
f^{(i)}(\mathbf{x}) &\sim g(\mathbf{x}) + \mathcal{GP}(\mathbf{0}, k^{(i)}(\mathbf{x}, \mathbf{x}')) \\
f^{(i)}(\mathbf{x}) &\sim \mathcal{GP}(\mathbf{0}, k^{(i)}(\mathbf{x}, \mathbf{x}') + k_g(\mathbf{x}, \mathbf{x}'))\\
\end{align}


A sensible approach would be to model this using a hierarchical set of Gaussian processes as they allow us to robustly propogate uncertainty through the whole hierarchical model. We've defined the group function as a Gaussian process with a covariance function defined by the kernel kg. As we can easily add Gaussian processes together we can add group functionality to individual Gaussian processes that model the additional functionality related to the individual signals. 

So in summary, we model the individual time series using a Gaussian process, using two covariance functions which seperately describe the individual behaviour and the behaviour that is shared across all time series.

# BUT!

$${\Huge \mathcal{O}(n^3r^2)}$$

The issue with this model is it doesn't scale well. Like normal Gaussian processes it scales cubically with the size of the data as well as quadratically with the number of individual time series, which for the above case was three.

So to realistically apply this model environmnetal dataset which are typically fairly large we need to find a way to speed that up. So we use variational approaches from the Gaussian process literature to make this hierarchical model usable. I won't go into details but I'll hopefully provide some inuition as to how it works...

In [ ]:
fig3, gp, sgp, x, opt_gp, opt_sgp = plot_fig3()
fig3

What does this look like? Well say we're fitting these data in black. As a standard Gaussian process fits it uses all the data available to it, whereas in Sparse Gaussian processes we fit using on a set of so-called 'inducing points' which in this case are only ten, compared to 100 in the standard GP. This low rank approximation caused by using less points speeds up the fitting of Gaussian processes enormously and allows us to fit bigger amounts of data, whilst still producing good estimates of uncertainty. You can see as well that the location of the inducing variables are not fixed and we are optimising there location as we fit the GP.

In [ ]:
fit_fig3(fig3, gp, sgp, x, opt_gp, opt_sgp);

# Sparse hierarchical Gaussian processes

\begin{align}
f^{(i)}(\mathbf{x}) \sim \mathcal{GP}(\mathbf{0}, k^{(i)}(\mathbf{x}, \mathbf{x}') + k_g(\mathbf{x}, \mathbf{x}'))\\
\end{align}


In [ ]:
# Hide, run initially
da = xr.open_dataarray('./../data/tas-ssp245.nc') - 273.15

fig4 = go.Figure()
for i in range(4):
    fig4.add_trace(
        go.Scatter(
            x=da.time,
            y=da.isel(realisation=i).values,
            name=str(da.realisation[i].values)[:-8]
        ))
fig4.update_layout(
    yaxis_title='Global mean surface temp. (°C)',
    width=full_fig_width,
    height=full_fig_height * 0.85,
    margin=dict(t=5))
fig4.show(config=config)

We can apply the same sparse methods to the hierarchical model we defined earlier to allow us to fit larger data that we see a lot of in the environmental sciences.

Let's take a climate example. These are four surface temperature projections out to the end of the century for central England taken from ssp245 simulations which is a middle of the road emissions scenario. These are all from seperate climate models. As I said earlier the trend across all the models should on the whole be similar, as should the seasonal cycle, but there'll be variation due to the differences in the model and the inherent chaos of the Earth system.

In [ ]:
fig5, shgp, da, norm_vals = plot_fig5()
fig5

What I'm showing here are 4 of the models on the right hand side, with their projections of surface temperature at the end of the century in the black dots. The fit of the hierarhical GP model is the coloured line, which is the mean and the shading is a 95% credible interval. This left hand side is the latent function aka the estimate of the signal that underlies all the models. These gray lines just show the means of the four models. What we see now is the prior of the model, before anything has been fit.

But as I fit the model, we constrain the posterior of both the individual model fits and therefore we constrain the latent posterior. I should say that this is fitting in real time for non compiled code, so we're fitting over 100000 datapoints which is something we wouldn't be able to do at this speed with a non sparse GP model. But I won't make you sit through the whole fitting!

In [ ]:
fit_fig5(fig5, shgp, da, norm_vals, n_iters=10)

Rather than watching that compile for 100+ steps this is what the final fit looks like. We've fit the more wiggly signal of each climate model whilst capturing the smoother latent trend which we interpret as the most likely model, in other words that's our best estimate of surface temperature projections for central england under the SSP245 scenario. It's still not as smooth as we would expect but that's due to the fact we've only used 4 models. When we do this with the entire the nearly 200 simulations run by CMIP6 models, the latent function is much smoother

# What next?

* Applications for air quality - averaging across low cost sensors
* Climate model ensembling

<center><img src="assets/GMST_scenario_comparison_with_1sd.png" width="700"></center>


As with all these talk, that was a whistle stop tour of how we can this scalable GP model we've developed and how we can use it in many environmental applications to model hierarchical systems and to specifically extract the latent behaviour of a system.

One ongoing collaboration which we're just writing up an extension of the climate example I just gave. The GP model I've described here is used in a larger Bayesian framework to probabilistically ensemble climate models. In that framework we use scoring metrics, such as the Kernel Stien discrepancy, to determine model skill. From that we can construct the barycentre, which is essentially the probablistic of all the climate models. This is what's shown in this figure. Each coloured line is the most likely projection pathway for the major emissions scenario from CMIP6.

Alongside this another application we're exploring is the to model air quality from multiple low cost sensors across a city. It's a very flexible tool and I'm definitely excited to extend it and see what else we can do with it.

Lastly I just wanted to highlight that this is all openly avaialable code. The Bayesian ensembling package which is functioning and nearly finished allows you to probabilistically ensemble climate model output in six lines of code. That's what the figure above was produced from.  We're hpoing to release the fully functioning package by the end of the year. The more methodological Sparse hierachical GP code is also available with a couple of air quality and climate examples.

# Open source

* https://github.com/mattramos/bayesian_ensembling
* https://github.com/mattramos/SparseHGP
* Papers on arxiv shortly!

### Code snippet from Bayesian ensemblings package
```
hist_models, fore_models = load_model_data(dir='data/gmst/ssp119')
hist_models.fit(model=es.SHGP())
weight_function = es.KSDWeight()
ksd_weights = weight_function(hist_models, observations)
ensemble_method = es.Barycentre()
barycentre = ensemble_method(fore_models, weights_119)
```